## Scalability Testing

This notebook is for testing purposes.

In [1]:
import random

from config import Color
from helpers import clean
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigNovis,
    ConfigQGC,
    Gazebo,
    NoVisualizer,
    Simulator,
)

clean()


## Simulation Configuration

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=90) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

gcs_colors=[
    Color.RED,
    Color.ORANGE,
    Color.GREEN,
    Color.BLUE
    ]

n_uavs_per_gcs = 60
side_len = 10
altitude = 5
max_delay = 3 #sec

base_homes = ENUPose.list([
    (i * 50, j * 3 * side_len, 0, 0)
    for i in range(len(gcs_colors))
    for j in range(n_uavs_per_gcs)
])

base_paths = [
    Plan.create_square_path(side_len=side_len, alt=altitude,heading=0) 
    for _ in base_homes
    ]

colors = [color for color in gcs_colors for _ in range(n_uavs_per_gcs)]

msn_delays=[random.randint(0, max_delay) for _ in base_homes]


## Assign vehicles to GCS (by color)
gcs_names = [f"{color.name}_{color.emoji}" for color in gcs_colors]
gcs_sysids = [
    list(range(i * n_uavs_per_gcs + 1, (i + 1) * n_uavs_per_gcs + 1)) 
    for i in range(len(gcs_colors))
    ]

## Gazebo Congiguration

In [3]:
gaz_config = ConfigGazebo(origin = enu_origin,
                          world_path="simulator/gazebo/worlds/runway3.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
#gaz_config.show()

gaz= Gazebo(gaz_config,gra_origin)

## QGroundControl Configuration

In [4]:
qgc_config = ConfigQGC(origin = gra_origin)

for path,home,color,delay in zip(base_paths,base_homes,colors,msn_delays):
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay)
qgc_config.show()

qgc=QGC(qgc_config)

# No Visualizer

In [5]:
novis_config = ConfigNovis(origin=gra_origin)
for home in base_homes:
    novis_config.add(base_home=home)
novis=NoVisualizer(novis_config)

## Launch Simulator

In [6]:
simulator = Simulator(
	visualizers=[novis],
	gcs_names=gcs_names,
	gcs_sysids=gcs_sysids,
	missions=[veh.mission for veh in qgc_config.vehicles],
	terminals=['gcs'],
	verbose=1,
)

orac = simulator.launch()

18:50:22 - Oracle ⚪ - INFO - 🙈 Running without visualization.
18:50:22 - Oracle ⚪ - INFO - 🚀 GCS RED_🟥 launched (PID 441450)
18:50:22 - Oracle ⚪ - INFO - 🚀 GCS ORANGE_🟧 launched (PID 441451)
18:50:22 - Oracle ⚪ - INFO - 🚀 GCS GREEN_🟩 launched (PID 441452)
18:50:22 - Oracle ⚪ - INFO - 🚀 GCS BLUE_🟦 launched (PID 441453)
18:50:22 - Oracle ⚪ - INFO - ⏳ Waiting for GCS processes to launch vehicles...
18:50:27 - Oracle ⚪ - INFO - 🔗 Starting Oracle connections to vehicles...
18:50:32 - Oracle ⚪ - INFO - 🔗 UAV logic 21 is connected to Oracle ⚪
18:50:32 - Oracle ⚪ - INFO - 🔗 UAV logic 1 is connected to Oracle ⚪
18:50:32 - Oracle ⚪ - INFO - 🔗 UAV logic 2 is connected to Oracle ⚪
18:50:32 - Oracle ⚪ - INFO - 🔗 UAV logic 9 is connected to Oracle ⚪
18:50:32 - Oracle ⚪ - INFO - 🔗 UAV logic 22 is connected to Oracle ⚪
18:50:32 - Oracle ⚪ - INFO - 🔗 UAV logic 11 is connected to Oracle ⚪
18:50:32 - Oracle ⚪ - INFO - 🔗 UAV logic 10 is connected to Oracle ⚪
18:50:32 - Oracle ⚪ - INFO - 🔗 UAV logic 5 is c

## Oracle checking

In [7]:
orac.run()

18:50:44 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 240 vehicles
18:50:44 - Oracle ⚪ - INFO - Monitoring 4 GCS processes
18:52:05 - Oracle ⚪ - INFO - Received message 'DONE' from GCS GREEN_🟩
18:52:05 - Oracle ⚪ - INFO - GCS GREEN_🟩 removal complete. Remaining connections: 180
18:52:05 - Oracle ⚪ - INFO - Received message 'DONE' from GCS ORANGE_🟧
18:52:05 - Oracle ⚪ - INFO - GCS ORANGE_🟧 removal complete. Remaining connections: 120
18:52:06 - Oracle ⚪ - INFO - Received message 'DONE' from GCS BLUE_🟦
18:52:06 - Oracle ⚪ - INFO - GCS BLUE_🟦 removal complete. Remaining connections: 60
18:52:06 - Oracle ⚪ - INFO - Received message 'DONE' from GCS RED_🟥
18:52:06 - Oracle ⚪ - INFO - GCS RED_🟥 removal complete. Remaining connections: 0
18:52:06 - Oracle ⚪ - INFO - ✅ Main monitoring loop completed - all connections closed
18:52:06 - Oracle ⚪ - INFO - 🎉 Oracle shutdown complete!
